In [2]:
## import library
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline
import plotly.express as px
pd.set_option('display.max_columns',None)
import warnings
warnings.filterwarnings("ignore")
from sklearn.preprocessing import OrdinalEncoder
from sklearn.ensemble import RandomForestRegressor,GradientBoostingRegressor
from sklearn.inspection import permutation_importance
from sklearn.model_selection import train_test_split
from sklearn.linear_model import Lasso
from sklearn.preprocessing import StandardScaler

In [10]:
df=pd.read_csv('/content/data_after_eda_Bivariate_and_multivariate_analysis')
df.head()

,flight_month,is_hoiliday,day_of_week,airline,class,departure_city,departure_time,arrival_city,arrival_time,duration,price,Stops,price_category
0,6,0,0,SpiceJet,economy,Delhi,1135,Mumbai,1265,130,6013,0,Medium
1,6,0,0,SpiceJet,economy,Delhi,380,Mumbai,520,140,6013,0,Medium
2,6,0,0,AirAsia,economy,Delhi,265,Mumbai,395,130,6016,0,Medium
3,6,0,0,Vistara,economy,Delhi,620,Mumbai,755,135,6015,0,Medium
4,6,0,0,Vistara,economy,Delhi,530,Mumbai,670,140,6015,0,Medium


In [11]:
df.shape

(298133, 13)

In [12]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 298133 entries, 0 to 298132
Data columns (total 13 columns):
 #   Column          Non-Null Count   Dtype 
---  ------          --------------   ----- 
 0   flight_month    298133 non-null  int64 
 1   is_hoiliday     298133 non-null  int64 
 2   day_of_week     298133 non-null  int64 
 3   airline         298133 non-null  object
 4   class           298133 non-null  object
 5   departure_city  298133 non-null  object
 6   departure_time  298133 non-null  int64 
 7   arrival_city    298133 non-null  object
 8   arrival_time    298133 non-null  int64 
 9   duration        298133 non-null  int64 
 10  price           298133 non-null  int64 
 11  Stops           298133 non-null  int64 
 12  price_category  298133 non-null  object
dtypes: int64(8), object(5)
memory usage: 29.6+ MB


In [13]:
df.select_dtypes('O').columns

Index(['airline', 'class', 'departure_city', 'arrival_city', 'price_category'], dtype='object')

In [14]:
data=df.copy()

In [15]:
category = df.select_dtypes('O').columns

In [16]:
custom_mappings = {
    'class': [['economy', 'business']]  # This will encode 'economy' as 0, 'business' as 1
}

for col in category:
    if col in custom_mappings:
        encoder = OrdinalEncoder(categories=custom_mappings[col])
    else:
        encoder = OrdinalEncoder()
    data[col] = encoder.fit_transform(data[[col]])
    print(f"{col}: {encoder.categories_}")

airline: [array(['Air India', 'AirAsia', 'GO FIRST', 'Indigo', 'SpiceJet',
       'StarAir', 'Trujet', 'Vistara'], dtype=object)]
class: [array(['economy', 'business'], dtype=object)]
departure_city: [array(['Bangalore', 'Chennai', 'Delhi', 'Hyderabad', 'Kolkata', 'Mumbai'],
      dtype=object)]
arrival_city: [array(['Bangalore', 'Chennai', 'Delhi', 'Hyderabad', 'Kolkata', 'Mumbai'],
      dtype=object)]
price_category: [array(['High', 'Low', 'Luxury', 'Medium', 'Very High'], dtype=object)]


In [17]:
data.sample(10)

,flight_month,is_hoiliday,day_of_week,airline,class,departure_city,departure_time,arrival_city,arrival_time,duration,price,Stops,price_category
34195,8,0,6,0.0,0.0,2.0,640,3.0,1295,655,4251,1,1.0
234591,7,0,0,0.0,1.0,5.0,1140,4.0,870,1170,50471,1,4.0
263214,7,0,3,0.0,1.0,4.0,1230,5.0,675,885,51025,1,2.0
44228,7,0,1,0.0,0.0,5.0,305,2.0,1005,700,23183,1,4.0
54502,7,0,5,1.0,0.0,5.0,285,0.0,520,235,5152,1,3.0
288224,8,0,4,7.0,1.0,1.0,1230,2.0,515,725,54242,1,2.0
91726,8,0,4,0.0,0.0,0.0,1150,2.0,645,935,4337,1,1.0
217915,7,0,3,0.0,1.0,2.0,290,3.0,1275,985,48119,1,4.0
166349,7,0,6,1.0,0.0,3.0,860,0.0,930,70,1794,0,1.0
57249,7,0,5,7.0,0.0,5.0,1255,0.0,650,835,6515,1,0.0


In [18]:
X=data.drop(columns='price')
y=data['price']

#`we are using all 8 technique of feature selection for real estate data feature selection `
##`1.> Correlation Analysis`

In [19]:
# Compute correlation matrix
corr_matrix =  data.corr().round(2)

# Convert to long format
corr_long = corr_matrix.reset_index().melt(id_vars='index')
corr_long.columns = ['Variable1', 'Variable2', 'Correlation']

# Plot heatmap
fig = px.imshow(
    corr_matrix,
    text_auto=True,
    color_continuous_scale='RdBu',
    aspect='auto',
    title='Correlation Heatmap (Numerical Features)'
)

fig.update_layout(
    width=800,
    height=600,
    xaxis_title='Variables',
    yaxis_title='Variables'
)

fig.show()

In [20]:
f1_df=data.corr()['price'].to_frame().reset_index().rename(columns={'index':'columns','price':'corr_coef'}).sort_values(by='corr_coef',ascending=False)
f1_df

,columns,corr_coef
10,price,1.000000
4,class,0.938969
12,price_category,0.271636
3,airline,0.242481
9,duration,0.215724
11,Stops,0.122064
8,arrival_time,0.077166
6,departure_time,0.032030
1,is_hoiliday,0.012178
7,arrival_city,0.005707


#`2.>- Embedded feature selection through random forest feature importance`

In [21]:
rfr=RandomForestRegressor(n_estimators=200)
model=rfr.fit(X,y)

In [22]:
f2_df=pd.DataFrame({'columns':X.columns,'feature_imp_rf':model.feature_importances_}).sort_values(by='feature_imp_rf',ascending=False)
f2_df

,columns,feature_imp_rf
4,class,0.884357
11,price_category,0.074046
9,duration,0.015403
6,departure_time,0.004495
8,arrival_time,0.004483
7,arrival_city,0.004352
5,departure_city,0.004322
3,airline,0.003138
2,day_of_week,0.002346
0,flight_month,0.001955


In [23]:
rfr=GradientBoostingRegressor(n_estimators=800)
model=rfr.fit(X,y)

In [24]:
f3_df=pd.DataFrame({'columns':X.columns,'feature_imp_gd':model.feature_importances_}).sort_values(by='feature_imp_gd',ascending=False)
f3_df

,columns,feature_imp_gd
4,class,0.902374
11,price_category,0.074914
9,duration,0.012801
3,airline,0.003363
7,arrival_city,0.001401
5,departure_city,0.001172
0,flight_month,0.001146
6,departure_time,0.001112
8,arrival_time,0.000844
10,Stops,0.000700


#`4.> permutation_importance`

In [25]:
X_train,X_test,y_train,y_test= train_test_split(X,y,test_size=0.2,random_state=42)
## baseline model
rfr=RandomForestRegressor(n_estimators=100,random_state=42)
model=rfr.fit(X_train,y_train)
permutation_imp=permutation_importance(model,X_test,y_test,n_repeats=30,random_state=42)
permutation_imp

{'importances_mean': array([5.33043009e-03, 1.10149221e-04, 2.12413923e-03, 1.92255001e-02,
        1.99312829e+00, 1.88875303e-02, 7.01925940e-03, 1.99784455e-02,
        5.77809828e-03, 2.68541923e-02, 3.53455836e-03, 1.29307805e-01]),
 'importances_std': array([2.51181738e-04, 1.50139047e-05, 1.45475404e-04, 3.55690181e-04,
        8.06522525e-03, 2.67360511e-04, 1.46276425e-04, 2.74546544e-04,
        1.33887704e-04, 4.11867813e-04, 1.24122185e-04, 9.15431401e-04]),
 'importances': array([[5.17641378e-03, 5.43215372e-03, 5.22357286e-03, 5.54008845e-03,
         5.19271428e-03, 4.85951322e-03, 5.33895959e-03, 5.60596631e-03,
         5.49211549e-03, 5.24641611e-03, 4.92800896e-03, 5.26603560e-03,
         5.66348760e-03, 5.69104689e-03, 5.17844617e-03, 4.89686916e-03,
         5.27378263e-03, 5.31496939e-03, 5.34225458e-03, 5.15374968e-03,
         5.58979221e-03, 5.81799032e-03, 5.61352531e-03, 5.13534257e-03,
         5.18403337e-03, 5.37800452e-03, 5.65977492e-03, 5.00155293e-03,

In [26]:
f4_df=pd.DataFrame({'columns':X.columns,'feature_imp_permutation':permutation_imp.importances_mean}).sort_values(by='feature_imp_permutation',ascending=False)
f4_df

,columns,feature_imp_permutation
4,class,1.993128
11,price_category,0.129308
9,duration,0.026854
7,arrival_city,0.019978
3,airline,0.019226
5,departure_city,0.018888
6,departure_time,0.007019
8,arrival_time,0.005778
0,flight_month,0.005330
10,Stops,0.003535


#`5.> lasso feature imp.`

In [27]:
scaler=StandardScaler()
X_scaled=scaler.fit_transform(X)
scaler_y = StandardScaler()
y_scaled = scaler_y.fit_transform(y.values.reshape(-1, 1)).ravel()
reg=Lasso(alpha=0.01,random_state=42)
model=reg.fit(X_scaled,y_scaled)
F5_df=pd.DataFrame({'columns':X.columns,'feature_imp_lasso':model.coef_}).sort_values(by='feature_imp_lasso',ascending=False)
F5_df

,columns,feature_imp_lasso
4,class,0.929180
10,Stops,0.093536
3,airline,0.065511
9,duration,0.022481
8,arrival_time,0.006381
7,arrival_city,0.000903
1,is_hoiliday,-0.000000
2,day_of_week,-0.000000
6,departure_time,-0.000000
5,departure_city,0.000000


#`6.> Recursive forward elimination`

In [28]:
from sklearn.feature_selection import RFE
## baseline model
rfr=RandomForestRegressor()
rfe=RFE(rfr,n_features_to_select=X.shape[0],step=1)
model=rfe.fit(X,y)
selected_feature=X.columns[model.support_]
selected_coef=model.estimator_.feature_importances_

In [29]:
F6_df=pd.DataFrame({'columns':selected_feature,'score_rfe':selected_coef}).sort_values(by='score_rfe',ascending=False)
F6_df

,columns,score_rfe
4,class,0.884361
11,price_category,0.074063
9,duration,0.015368
6,departure_time,0.004489
8,arrival_time,0.004478
5,departure_city,0.004355
7,arrival_city,0.004350
3,airline,0.003143
2,day_of_week,0.002337
0,flight_month,0.001957


#`7.> Linear reg.`

In [30]:
from sklearn.linear_model import LinearRegression
# from sklearn.preprocessing import StandardScaler
# sc=StandardScaler()
# X_scaled=sc.fit_transform(X)
## baseline model
lr=LinearRegression()
lr.fit(X_scaled,y_scaled)
F7_df=pd.DataFrame({'columns':X.columns,'feature_imp_linear_reg':lr.coef_}).sort_values(by='feature_imp_linear_reg',ascending=False)
F7_df

,columns,feature_imp_linear_reg
4,class,0.942274
10,Stops,0.099215
3,airline,0.073540
9,duration,0.029407
8,arrival_time,0.013995
7,arrival_city,0.013729
5,departure_city,0.010858
6,departure_time,-0.002596
1,is_hoiliday,-0.005508
2,day_of_week,-0.006956


#`SHAP`

In [31]:
# ! pip install shap

In [33]:
# import shap
# ## baseline model
# rfr=RandomForestRegressor(n_estimators=100)
# rfr.fit(X,y)

In [34]:
# shap_model=shap.TreeExplainer(rfr)

In [35]:
# sample_X = X.sample(n=10000, random_state=42)

In [36]:
# shap_value=shap_model.shap_values(sample_X)
# ## we have to sum the values across all sample to get overall feature imp.
# feature_imp=np.abs(shap_value).mean(axis=0)
# F8_df=pd.DataFrame({'columns':X.columns,'shap_rfe':feature_imp}).sort_values(by='shap_rfe',ascending=False)
# F8_df

In [37]:
final_Feature_df = f1_df.merge(f2_df, on='columns') \
                        .merge(f3_df, on='columns') \
                        .merge(f4_df, on='columns') \
                        .merge(F5_df, on='columns') \
                        .merge(F6_df, on='columns') \
                        .merge(F7_df, on='columns') \
                        .set_index('columns')

In [38]:
final_Feature_df

,corr_coef,feature_imp_rf,feature_imp_gd,feature_imp_permutation,feature_imp_lasso,score_rfe,feature_imp_linear_reg
columns,,,,,,,
class,0.938969,0.884357,0.902374,1.993128,0.929180,0.884361,0.942274
price_category,0.271636,0.074046,0.074914,0.129308,-0.047663,0.074063,-0.064828
airline,0.242481,0.003138,0.003363,0.019226,0.065511,0.003143,0.073540
duration,0.215724,0.015403,0.012801,0.026854,0.022481,0.015368,0.029407
Stops,0.122064,0.000919,0.000700,0.003535,0.093536,0.000915,0.099215
arrival_time,0.077166,0.004483,0.000844,0.005778,0.006381,0.004478,0.013995
departure_time,0.032030,0.004495,0.001112,0.007019,-0.000000,0.004489,-0.002596
is_hoiliday,0.012178,0.000185,0.000005,0.000110,-0.000000,0.000186,-0.005508
arrival_city,0.005707,0.004352,0.001401,0.019978,0.000903,0.004350,0.013729


In [39]:
final_Feature_df=final_Feature_df.divide(final_Feature_df.sum(axis=0),axis=1)
final_Feature_df

,corr_coef,feature_imp_rf,feature_imp_gd,feature_imp_permutation,feature_imp_lasso,score_rfe,feature_imp_linear_reg
columns,,,,,,,
class,0.509786,0.884357,0.902374,0.893267,0.916050,0.884361,0.910290
price_category,0.147477,0.074046,0.074914,0.057952,-0.046989,0.074063,-0.062627
airline,0.131648,0.003138,0.003363,0.008616,0.064585,0.003143,0.071044
duration,0.117121,0.015403,0.012801,0.012035,0.022163,0.015368,0.028409
Stops,0.066271,0.000919,0.000700,0.001584,0.092215,0.000915,0.095847
arrival_time,0.041895,0.004483,0.000844,0.002590,0.006291,0.004478,0.013520
departure_time,0.017390,0.004495,0.001112,0.003146,-0.000000,0.004489,-0.002508
is_hoiliday,0.006612,0.000185,0.000005,0.000049,-0.000000,0.000186,-0.005321
arrival_city,0.003099,0.004352,0.001401,0.008954,0.000890,0.004350,0.013263


In [45]:
feature_selection_data=final_Feature_df[['feature_imp_rf','feature_imp_gd','feature_imp_permutation','score_rfe']].mean(axis=1).sort_values(ascending=False)

##`Train and test WITH columns`

In [47]:
feature_selection_data.to_csv('feature_selection_data')

In [42]:
df[df['Stops']==2]['price'].mean()

np.float64(13975.249558232932)

In [43]:
df[df['Stops']==1]['price'].mean()

np.float64(23220.11283893943)

In [44]:
df[df['Stops']==0]['price'].mean()

np.float64(9493.462878703807)